In [45]:
# =========================
# Builds an ARDL-style regression:
#   Δln(price) ~ lags(Δln(price)) + lags(Δln(mortgage)) + lag(policy rate) + lag(CPI inflation) + lag(Δln(wage)) + month FE
# Uses HAC (Newey–West) standard errors
# =========================

import pandas as pd
import numpy as np
import re
import statsmodels.api as sm
from statsmodels.stats.diagnostic import acorr_ljungbox

PATH = "ИЖК_Цены_Ставка_Зарплата_CPI_2009-2024_горизонтально.xlsx"
SHEET = "horizontal"

raw = pd.read_excel(PATH, sheet_name=SHEET, header=None)

# -------- 2) Helper: find row by label in column A --------
def find_row(label: str) -> int:
    col0 = raw.iloc[:, 0].astype(str).str.strip()
    idx = col0[col0 == label].index
    if len(idx) == 0:
        # try a contains match as fallback
        idx = col0[col0.str.contains(re.escape(label), case=False, na=False)].index
    if len(idx) == 0:
        raise ValueError(f"Row with label '{label}' not found in column A.")
    return int(idx[0])

# -------- 3) Read header dates --------
date_keys = raw.iloc[0, 1:].astype(str).tolist()
dates = pd.to_datetime(date_keys, format="%Y-%m", errors="coerce")
if dates.isna().any():
    bad = [date_keys[i] for i, d in enumerate(dates) if pd.isna(d)]
    raise ValueError(f"Some date headers could not be parsed: {bad[:10]}")

# -------- 4) Extract series from rows --------
# Labels used in your file (adjust if your row names differ)
r_price   = find_row("price_rub_m2")
r_mort    = find_row("moscow_flow_mn_rub")
r_wage    = find_row("wage_nominal_rub_moscow")
r_rate    = find_row("policy_rate_monthly_avg_pct")
r_cpi     = find_row("cpi_mom_index_pct")

price = pd.to_numeric(raw.iloc[r_price, 1:], errors="coerce").to_numpy()
mort  = pd.to_numeric(raw.iloc[r_mort,  1:], errors="coerce").to_numpy()
wage  = pd.to_numeric(raw.iloc[r_wage,  1:], errors="coerce").to_numpy()
rate  = pd.to_numeric(raw.iloc[r_rate,  1:], errors="coerce").to_numpy()
cpi   = pd.to_numeric(raw.iloc[r_cpi,   1:], errors="coerce").to_numpy()

df = pd.DataFrame({
    "date": dates,
    "price_rub_m2": price,
    "mortgage_mn_rub": mort,
    "wage_moscow_rub": wage,
    "policy_rate_avg_pct": rate,
    "cpi_mom_index_pct": cpi
}).sort_values("date").reset_index(drop=True)

# -------- 5) Transformations --------
df["infl_mom"] = df["cpi_mom_index_pct"] / 100.0 - 1.0

df["ln_price"]    = np.log(df["price_rub_m2"])
df["ln_mortgage"] = np.log(df["mortgage_mn_rub"])
df["ln_wage"]     = np.log(df["wage_moscow_rub"])

df["dln_price"]    = df["ln_price"].diff()
df["dln_mortgage"] = df["ln_mortgage"].diff()
df["dln_wage"]     = df["ln_wage"].diff()

# Lags
for k in [1, 2]:
    df[f"dln_price_l{k}"]    = df["dln_price"].shift(k)
    df[f"dln_mortgage_l{k}"] = df["dln_mortgage"].shift(k)

df["policy_rate_l1"] = df["policy_rate_avg_pct"].shift(1)
df["infl_mom_l1"]    = df["infl_mom"].shift(1)
df["dln_wage_l1"]    = df["dln_wage"].shift(1)

# Month fixed effects
df["month"] = df["date"].dt.month
month_fe = pd.get_dummies(df["month"], prefix="m", drop_first=True)

# -------- 6) Build regression dataset --------
y = df["dln_price"]

X = pd.concat([
    df[[
        "dln_price_l1", "dln_price_l2",
        "dln_mortgage_l1", "dln_mortgage_l2",
        "policy_rate_l1",
        "infl_mom_l1",
        "dln_wage_l1"
    ]],
    month_fe
], axis=1)

reg = pd.concat([df[["date"]], y, X], axis=1).dropna().copy()

Y = reg["dln_price"].astype(float)
Xmat = sm.add_constant(reg.drop(columns=["date", "dln_price"]).astype(float), has_constant="add")

# -------- 7) Estimate with HAC standard errors --------
model = sm.OLS(Y, Xmat).fit(cov_type="HAC", cov_kwds={"maxlags": 12})

print(model.summary())

# -------- 8) Diagnostics: residual autocorrelation --------
lb = acorr_ljungbox(model.resid, lags=[6, 12], return_df=True)
print("\nLjung–Box test (residual autocorrelation):")
print(lb)

# -------- 9) Save model-ready dataset and results (optional) --------
out_path = "initial_model_results.xlsx"
coef_table = pd.DataFrame({
    "coef": model.params,
    "se_HAC": model.bse,
    "t": model.tvalues,
    "p": model.pvalues
})

with pd.ExcelWriter(out_path, engine="openpyxl") as writer:
    reg.to_excel(writer, sheet_name="data_used", index=False)
    coef_table.to_excel(writer, sheet_name="coefficients")
    lb.to_excel(writer, sheet_name="ljung_box")

print("\nSaved:", out_path)

                            OLS Regression Results                            
Dep. Variable:              dln_price   R-squared:                       0.161
Model:                            OLS   Adj. R-squared:                  0.038
Method:                 Least Squares   F-statistic:                     5.687
Date:                Mon, 23 Feb 2026   Prob (F-statistic):           1.27e-09
Time:                        13:48:55   Log-Likelihood:                 372.00
No. Observations:                 142   AIC:                            -706.0
Df Residuals:                     123   BIC:                            -649.8
Df Model:                          18                                         
Covariance Type:                  HAC                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.0144      0.016     